In [1]:
import pandas as pd
import requests
import time
from datetime import datetime
import json
import logging
import os

logging.basicConfig(filename='12_nbconvert_proxy.log', level=logging.INFO)

# Request rate	4 lookups / min
# Daily quota	500 lookups / day
# Monthly quota	15.5 K lookups / month

sleep = 200  # sleep 200 seconds after each call

headers = {
    "accept": "application/json",
    # https://www.virustotal.com/gui/user/lsys/apikey
}

In [2]:
from requests.exceptions import ProxyError, ReadTimeout, ConnectTimeout
from fp.fp import FreeProxy

def get_proxy():
    try:
        proxy = FreeProxy(rand=True).get()
        scheme_proxy_map = {
            'http': proxy,
            'https': proxy,
        }
        response = requests.get('https://ip.oxylabs.io/ip', proxies=scheme_proxy_map, timeout=10, verify=False)
        #print(response.text)
        if len(response.text.split()) == 1:
            return proxy
        else:
            return None
    #except (ProxyError, ReadTimeout, ConnectTimeout) as error:
    except Exception as error:
        print('Unable to connect to the proxy: ', error)
        return None


In [3]:
while True:
    print('Get proxy...')
    proxy = get_proxy()
    if proxy:
        print(proxy)
        break
    time.sleep(5)

Get proxy...


/home/jupyter/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '3.96.207.125'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Get proxy...


/home/jupyter/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.125.218.203'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Get proxy...


/home/jupyter/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '13.125.218.203'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Get proxy...


/home/jupyter/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '186.121.235.222'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


http://186.121.235.222:8080


In [4]:
scheme_proxy_map = {
    'http': proxy,
    'https': proxy,
}

In [5]:
chunk = "data/chunk2.csv"
df = pd.read_csv(chunk)

for ix, row in df.iterrows():
    now = datetime.now().strftime("%d %b %Y %H:%M:%S")
    domain = row["private_domain"]
    return_code = row["return_code"]
    
    ## Move on if already queried
    if return_code == 200:
        logging.info(f"({1+ix}/{len(df)}, {now}): Return code: {int(return_code)} - {domain}")
        continue

    if os.path.exists(f"payloads_json/{domain}.json"):
        logging.info(f"({1+ix}/{len(df)}, {now}): Return code: {return_code} - {domain}")
        continue

    while True:
        try:
            ## Query
            # https://developers.virustotal.com/reference/domain-info
            vt_url = f"https://www.virustotal.com/api/v3/domains/{domain}"
            response = requests.get(vt_url, headers=headers, proxies=scheme_proxy_map, timeout=15, verify=False)
            break
        except Exception as e:
            logging.error(f'Error: {e}')
            # Get new proxy
            while True:
                print('Get proxy...')
                proxy = get_proxy()
                if proxy:
                    print(proxy)
                    break
                time.sleep(5)
            scheme_proxy_map = {
                'http': proxy,
                'https': proxy,
            }
    ## Save payload
    return_code = response.status_code 
    if return_code == 200:
        with open(f"payloads_json/{domain}.json", "wb") as f:
            f.write(response.content)

    ## Update csv to indicate row is already queried
    df.at[ix, "return_code"] = return_code

    logging.info(f"({1+ix}/{len(df)}, {now}): Return code: {return_code} - {domain}")

    time.sleep(sleep)

df.to_csv(chunk, index=False)

/home/jupyter/venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '186.121.235.222'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
df.to_csv(chunk, index=False)

In [7]:
df.query("return_code==200")

,private_domain,return_code
251,evanced.info,200.0
252,pinata.ai,200.0
253,rmine.co,200.0
254,replaymod.com,200.0
255,rhoadsenergy.com,200.0
...,...,...
6993,knfilters.com,200.0
6994,ambankwacoonline.com,200.0
6995,rewasd.com,200.0
6996,finduslocal.com,200.0
